In [2]:
# import packages
import pandas as pd
import numpy as np

import swifter
import great_expectations as ge

In [3]:
# import datasets that have been cleaned via VIF check
df_nac = pd.read_csv('/Users/eesoonhang/Desktop/capstone_data/train_cleaned_nac.csv', usecols=lambda x: x!= '*Unnamed*')
df_dac = pd.read_csv('/Users/eesoonhang/Desktop/capstone_data/train_cleaned_dac.csv', usecols=lambda x: x!= '*Unnamed*')
df_tac = pd.read_csv('/Users/eesoonhang/Desktop/capstone_data/train_cleaned_tac.csv', usecols=lambda x: x!= '*Unnamed*')
train_X = pd.concat([df_nac, df_dac, df_tac], axis=1)
train_X.shape

(304661, 63)

In [4]:
# import train_output, test_input, test_output
# Note: do ensure test_input has similar column as train_input
test_X = pd.read_csv('/Users/eesoonhang/Desktop/capstone_data/test_in_encodedToNacSeq.csv', usecols=train_X.columns)
print(test_X.columns)
train_y = pd.read_csv('/Users/eesoonhang/Desktop/capstone_data/train_out.csv', usecols=lambda x: x!= '*Unnamed*')
test_y = pd.read_csv('/Users/eesoonhang/Desktop/capstone_data/test_out.csv', usecols=lambda x: x!= '*Unnamed*')

# compressed output y
# transform target >=1 to 1
def normalize_output(x):
    return np.where(x.target>1, x.target/x.target, x.target)
    
# compressed y-variable
compressed_train_y = train_y.sum(axis=1).to_frame(name='target')
compressed_test_y = test_y.sum(axis=1).to_frame(name='target')
compressed_train_y.target = compressed_train_y.swifter.apply(normalize_output, axis=1)
compressed_test_y.target = compressed_test_y.swifter.apply(normalize_output, axis=1)

Index(['A', 'C', 'G', 'T', 'AA', 'AC', 'CA', 'CC', 'CG', 'GA', 'GG', 'GT',
       'TA', 'TC', 'TG', 'TT', 'AAA', 'AAC', 'AAG', 'ACC', 'ACG', 'ACT', 'AGA',
       'AGC', 'AGT', 'ATA', 'ATC', 'ATG', 'CAA', 'CAC', 'CAT', 'CCA', 'CCC',
       'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTT', 'GAC', 'GAT', 'GCA', 'GCT',
       'GGA', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG',
       'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TTA', 'TTC', 'TTT'],
      dtype='object')


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import math

def train_Model():
    
    print('scaling data')
    # apply scaler
    scaler = StandardScaler()
    train_X_scaled = scaler.fit_transform(train_X, compressed_train_y)
    train_X_scaled = pd.DataFrame(train_X_scaled, columns=train_X.columns)
    X_train, X_valid, Y_train, Y_valid = train_test_split(train_X_scaled, compressed_train_y, test_size = 0.5, random_state = 1234, stratify=compressed_train_y)
    
    print('fit model...')
        # instantiate model
    rfc = RandomForestClassifier(n_estimators=100)
    # Perform RandomForestClassification
    rfc.fit(X_train, Y_train.values.ravel())
    y_predict = rfc.predict(X_valid)
    
    print('evaluate model...')
    # evaluate model
    rmse = math.sqrt(mean_squared_error(Y_valid, y_predict))
    print('rmse: %4.2f' %rmse)
    
    score = rfc.score(X_valid, Y_valid)
    print('score: %4.2f' %score)
    
    return rmse, score, rfc, scaler

In [12]:
# train model
%time model = train_Model()

scaling data
fit model...
evaluate model...
rmse: 0.44
score: 0.81
CPU times: user 1min 8s, sys: 1.21 s, total: 1min 9s
Wall time: 1min 11s


In [15]:
# export the model
import pickle
pickle.dump(model[2], open('/Users/eesoonhang/Desktop/capstone_data/RFC_100trees_withVIF.pkl', 'wb'))
pickle.dump(model[3], open('/Users/eesoonhang/Desktop/capstone_data/scaler_VIFdata.pkl', 'wb'))

In [13]:
# evaluate accuracy
clf = model[2]
scaler = model[3]
test_X_scaled = scaler.transform(test_X)
clf.score(test_X_scaled, compressed_test_y)

0.7608333333333334